<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_hsj_MLP_SHAP_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 10.2 MB/s eta 0:00:00


In [3]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load trainset

In [4]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/multi/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385156 entries, 0 to 385155
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      385156 non-null  float64
 1   Tot Fwd Pkts       385156 non-null  float64
 2   Tot Bwd Pkts       385156 non-null  float64
 3   TotLen Fwd Pkts    385156 non-null  float64
 4   TotLen Bwd Pkts    385156 non-null  float64
 5   Fwd Pkt Len Max    385156 non-null  float64
 6   Fwd Pkt Len Min    385156 non-null  float64
 7   Fwd Pkt Len Mean   385156 non-null  float64
 8   Fwd Pkt Len Std    385156 non-null  float64
 9   Bwd Pkt Len Max    385156 non-null  float64
 10  Bwd Pkt Len Min    385156 non-null  float64
 11  Bwd Pkt Len Mean   385156 non-null  float64
 12  Bwd Pkt Len Std    385156 non-null  float64
 13  Flow Byts/s        385156 non-null  float64
 14  Flow Pkts/s        385156 non-null  float64
 15  Flow IAT Mean      385156 non-null  float64
 16  Fl

In [5]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385156 entries, 0 to 385155
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      385156 non-null  float64
 1   Tot Fwd Pkts       385156 non-null  float64
 2   Tot Bwd Pkts       385156 non-null  float64
 3   TotLen Fwd Pkts    385156 non-null  float64
 4   TotLen Bwd Pkts    385156 non-null  float64
 5   Fwd Pkt Len Max    385156 non-null  float64
 6   Fwd Pkt Len Min    385156 non-null  float64
 7   Fwd Pkt Len Mean   385156 non-null  float64
 8   Fwd Pkt Len Std    385156 non-null  float64
 9   Bwd Pkt Len Max    385156 non-null  float64
 10  Bwd Pkt Len Min    385156 non-null  float64
 11  Bwd Pkt Len Mean   385156 non-null  float64
 12  Bwd Pkt Len Std    385156 non-null  float64
 13  Flow Byts/s        385156 non-null  float64
 14  Flow Pkts/s        385156 non-null  float64
 15  Flow IAT Mean      385156 non-null  float64
 16  Fl

# Load adv samples

In [6]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [7]:
hsj_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/hsj/adv_hsj_target.csv'
#hsj_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/hsj/adv_hsj_target.csv'

In [8]:
hsj_df = pd.read_csv(hsj_path)

In [9]:
hsj_df.columns = feature_list
hsj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77203 entries, 0 to 77202
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      77203 non-null  float64
 1   Tot Fwd Pkts       77203 non-null  float64
 2   Tot Bwd Pkts       77203 non-null  float64
 3   TotLen Fwd Pkts    77203 non-null  float64
 4   TotLen Bwd Pkts    77203 non-null  float64
 5   Fwd Pkt Len Max    77203 non-null  float64
 6   Fwd Pkt Len Min    77203 non-null  float64
 7   Fwd Pkt Len Mean   77203 non-null  float64
 8   Fwd Pkt Len Std    77203 non-null  float64
 9   Bwd Pkt Len Max    77203 non-null  float64
 10  Bwd Pkt Len Min    77203 non-null  float64
 11  Bwd Pkt Len Mean   77203 non-null  float64
 12  Bwd Pkt Len Std    77203 non-null  float64
 13  Flow Byts/s        77203 non-null  float64
 14  Flow Pkts/s        77203 non-null  float64
 15  Flow IAT Mean      77203 non-null  float64
 16  Flow IAT Std       772

In [ ]:
#hsj_label = pd.read_csv(hsj_label_path)
#hsj_label.info()

# Load DL-based IDS model

In [11]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_multi.h5', compile=True)

# SHAP

In [13]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [32]:
hsj_1000 = hsj_df.head(1000)

In [33]:
shap_values = explainer.shap_values(hsj_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [34]:
print(shap_values[0])

[[ 0.          0.00371355  0.02583556 ... -0.02226725  0.01943251
   0.00468528]
 [ 0.          0.00382709  0.03309501 ...  0.          0.
  -0.02040464]
 [-0.06580376 -0.01609701  0.07522759 ...  0.01527657 -0.15340129
  -0.10001612]
 ...
 [ 0.         -0.02564431  0.04212916 ...  0.02594601  0.00581202
   0.03562748]
 [ 0.         -0.00733388  0.         ...  0.          0.00426713
   0.        ]
 [ 0.01304496  0.015048    0.0311654  ... -0.01907463 -0.0069883
  -0.00752316]]


In [36]:
type(shap_values[0])

numpy.ndarray

In [57]:
data_df = pd.DataFrame(shap_values[0])

In [60]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.000000,0.003714,0.025836,0.000000,0.025130,0.068690,0.000000,0.020516,0.040778,0.000000,...,0.004535,0.000000,0.008135,0.000000,0.000000,0.003866,0.018963,-0.022267,0.019433,0.004685
1,0.000000,0.003827,0.033095,0.000000,0.000000,0.072428,0.015280,0.075210,0.035304,0.016528,...,0.000000,0.011992,0.000000,0.018308,0.000000,-0.020093,-0.026236,0.000000,0.000000,-0.020405
2,-0.065804,-0.016097,0.075228,0.000000,0.005063,0.177326,0.025644,0.107967,0.357114,-0.009148,...,0.002266,0.008741,0.008404,-0.016454,0.000000,0.012872,-0.055721,0.015277,-0.153401,-0.100016
3,0.000000,0.033557,0.014961,-0.011775,-0.009682,0.286565,0.023302,0.150747,0.150969,0.013302,...,0.049484,0.000000,0.000000,0.011590,-0.013484,0.000000,0.005107,0.000000,0.000000,0.013351
4,-0.071150,0.013547,0.008769,-0.006781,-0.004302,0.309107,0.040704,0.127528,0.215590,0.002360,...,0.031409,0.000000,0.004243,-0.008165,0.022555,0.018825,-0.059504,0.004437,-0.130826,-0.054179


In [59]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_HSJ/1000_samples_[0].csv', index=False)

In [61]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.000000,0.003714,0.025836,0.000000,0.025130,0.068690,0.000000,0.020516,0.040778,0.000000,...,0.004535,0.000000,0.008135,0.000000,0.000000,0.003866,0.018963,-0.022267,0.019433,0.004685
1,0.000000,0.003827,0.033095,0.000000,0.000000,0.072428,0.015280,0.075210,0.035304,0.016528,...,0.000000,0.011992,0.000000,0.018308,0.000000,-0.020093,-0.026236,0.000000,0.000000,-0.020405
2,-0.065804,-0.016097,0.075228,0.000000,0.005063,0.177326,0.025644,0.107967,0.357114,-0.009148,...,0.002266,0.008741,0.008404,-0.016454,0.000000,0.012872,-0.055721,0.015277,-0.153401,-0.100016
3,0.000000,0.033557,0.014961,-0.011775,-0.009682,0.286565,0.023302,0.150747,0.150969,0.013302,...,0.049484,0.000000,0.000000,0.011590,-0.013484,0.000000,0.005107,0.000000,0.000000,0.013351
4,-0.071150,0.013547,0.008769,-0.006781,-0.004302,0.309107,0.040704,0.127528,0.215590,0.002360,...,0.031409,0.000000,0.004243,-0.008165,0.022555,0.018825,-0.059504,0.004437,-0.130826,-0.054179


In [72]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [73]:
print(data_dict)

{'Adv sample 0': ['Down/Up Ratio', 'Fwd Pkt Len Max', 'Flow Byts/s', 'Bwd Pkts/s', 'FIN Flag Cnt', 'Flow IAT Max', 'Fwd IAT Tot', 'Fwd Pkt Len Std', 'Fwd IAT Min', 'Bwd IAT Max'], 'Adv sample 1': ['Flow Byts/s', 'Pkt Len Std', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Max', 'Down/Up Ratio', 'Pkt Size Avg', 'ACK Flag Cnt', 'Fwd IAT Std', 'Fwd Seg Size Avg', 'Pkt Len Mean'], 'Adv sample 2': ['Fwd Pkt Len Std', 'Fwd Pkt Len Max', 'Fwd Seg Size Avg', 'Flow Byts/s', 'Fwd Pkt Len Mean', 'Pkt Len Min', 'Pkt Len Std', 'Bwd Header Len', 'Pkt Size Avg', 'Tot Bwd Pkts'], 'Adv sample 3': ['Fwd Pkt Len Max', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'SYN Flag Cnt', 'Pkt Size Avg', 'Flow Byts/s', 'Fwd Seg Size Avg', 'Pkt Len Min', 'Fwd Act Data Pkts', 'Pkt Len Std'], 'Adv sample 4': ['Fwd Pkt Len Max', 'Fwd Pkt Len Std', 'Fwd Seg Size Avg', 'Bwd Header Len', 'Fwd Pkt Len Mean', 'Flow Byts/s', 'Pkt Len Std', 'Pkt Len Min', 'Pkt Len Mean', 'Pkt Size Avg'], 'Adv sample 5': ['Init Fwd Win Byts', 'Pkt Len Min', 'SYN

In [71]:
len(shap_values[0])

1000

In [74]:
len(data_dict)

1000

# Extract local feature for each adv sample (77203 in total)

In [ ]:
for idx in range(0, 77203):
  # Create a DataFrame with the SHAP values and feature names
  df_shap_values = pd.DataFrame({'SHAP Values': exp[idx].values}, index=feature_list)
  df_shap_values.sort_values(by=['SHAP Values'], ascending=False, inplace=True)
  df_shap_values.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/hsj_Result/SHAP_ADVSamples_' + str(idx) + '.csv')
  print('Extracted local importance feature of the ' + str(idx) + 'adv sample.')

# Detection phase

In [67]:
import os
import pandas as pd

In [76]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,FIN Flag Cnt,1223
1,SYN Flag Cnt,1196
2,Init Fwd Win Byts,986
3,Flow Pkts/s,884
4,ACK Flag Cnt,808
5,Pkt Size Avg,767
6,Bwd Pkts/s,752
7,Down/Up Ratio,732
8,Pkt Len Mean,667
9,Pkt Len Min,588


## Detection rate (top 24) = 969/1000

In [77]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std']


In [78]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:969


## Detection rate (top 38) = 783/1000

In [79]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['FIN Flag Cnt', 'SYN Flag Cnt', 'Init Fwd Win Byts', 'Flow Pkts/s', 'ACK Flag Cnt', 'Pkt Size Avg', 'Bwd Pkts/s', 'Down/Up Ratio', 'Pkt Len Mean', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Pkt Len Max', 'Init Bwd Win Byts', 'Fwd Pkt Len Max', 'Fwd IAT Tot', 'Flow IAT Max', 'PSH Flag Cnt', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean', 'Pkt Len Std', 'Bwd IAT Tot', 'Idle Max', 'Fwd Seg Size Avg', 'Fwd IAT Std', 'Bwd PSH Flags', 'Flow Duration', 'Bwd IAT Std', 'Fwd Pkt Len Min', 'Flow IAT Std', 'Bwd IAT Max', 'Idle Mean', 'Fwd IAT Max', 'URG Flag Cnt', 'Idle Min', 'Flow IAT Mean', 'Fwd IAT Mean', 'Flow Byts/s', 'Bwd Pkt Len Mean']


In [80]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:783
